In [ ]:
def create_opensearch_provider_config():
    """
    Create EODAG provider configuration using OpenSearch plugins for Copernicus Dataspace
    
    EODAG Plugin Architecture:
    - SearchPlugin: eodag.plugins.search.qssearch.QueryStringSearch (OpenSearch)
    - DownloadPlugin: eodag.plugins.download.http.HTTPDownload
    - AuthPlugin: eodag.plugins.authentication.oauth.KeycloakOIDCPasswordAuth
    """
    
    config = {
        'cop_dataspace': {
            'priority': 1,
            'description': 'Copernicus Dataspace with OpenSearch API',
            
            # Search plugin configuration (OpenSearch)
            'search': {
                'plugin': 'QueryStringSearch',  # Built-in OpenSearch plugin
                'api_endpoint': 'https://catalogue.dataspace.copernicus.eu/resto/api/collections/{collection}/search.json',
                'need_auth': False,
                'timeout': 120,
                'ssl_verify': True,
                'pagination': {
                    'next_page_query_obj': '{{"page": {page}, "maxRecords": {items_per_page}}}',
                    'total_items_nb_key_path': '$.properties.totalResults',
                    'max_items_per_page': 1000
                },
                'discover_metadata': {
                    'auto_discovery': True,
                    'metadata_pattern': 'https://catalogue.dataspace.copernicus.eu/resto/api/collections/{collection}/describe.xml',
                    'search_param': 'q'
                },

                'literal_search_params': {
                    'format': 'json'
                },
                'constraints_file_url': 'https://catalogue.dataspace.copernicus.eu/resto/api/collections/describe.xml',
                'results_entry': 'features',
                'metadata_mapping': {
                    # Basic metadata mapping for OpenSearch
                    'uid': '$.id',
                    'title': '$.properties.title',
                    'geometry': '$.geometry',
                    'startTimeFromAscendingNode': '$.properties.startDate',
                    'completionTimeFromAscendingNode': '$.properties.completionDate',
                    'publicationDate': '$.properties.published',
                    'modificationDate': '$.properties.updated',
                    'platform': '$.properties.platform',
                    'instrument': '$.properties.instrument',
                    'productType': '$.properties.productType',
                    'processingLevel': '$.properties.processingLevel',
                    'orbitNumber': '$.properties.orbitNumber',
                    'relativeOrbitNumber': '$.properties.relativeOrbitNumber',
                    'orbitDirection': '$.properties.orbitDirection',
                    'cloudCover': '$.properties.cloudCover',
                    'sensorMode': '$.properties.sensorMode',
                    'polarizationChannels': '$.properties.polarisation',
                    'resolution': '$.properties.resolution',
                    'tileIdentifier': '$.properties.tileId',
                    'downloadLink': '$.properties.services.download.url',
                    'quicklook': '$.properties.quicklook',
                    'thumbnail': '$.properties.thumbnail'
                }
            },
            
            # Download plugin configuration
            'download': {
                'plugin': 'HTTPDownload',  # Built-in HTTP download plugin
                'extract': True,
                'archive_depth': 2,
                'ssl_verify': True,
                'timeout': 300,
                'chunk_size': 8192
            },
            
            # Authentication plugin configuration (matching OData config)
            'auth': {
                'plugin': 'KeycloakOIDCPasswordAuth',  # Keycloak OIDC plugin
                'matching_url': 'https://catalogue.dataspace.copernicus.eu',
                'oidc_config_url': 'https://identity.dataspace.copernicus.eu/auth/realms/CDSE/.well-known/openid-configuration',
                'client_id': 'cdse-public',
                'client_secret': None,
                'token_provision': 'qs',
                'token_qs_key': 'token',
                'auth_error_code': 401,
                'ssl_verify': True,
                'allowed_audiences': ["CLOUDFERRO_PUBLIC"],
                'credentials': {
                    'username': '${COPERNICUS_USERNAME}',
                    'password': '${COPERNICUS_PASSWORD}'
                }
            },
            
            # Complete product type mapping for all available collections
            'products': {
                # Sentinel-1 products
                'S1_SAR_RAW': {
                    'collection': 'SENTINEL-1',
                    'productType': 'RAW'
                },
                'S1_SAR_GRD': {
                    'collection': 'SENTINEL-1',
                    'productType': 'GRD'
                },
                'S1_SAR_GRD_COG': {
                    'collection': 'SENTINEL-1',
                    'productType': 'GRD-COG'
                },
                'S1_SAR_SLC': {
                    'collection': 'SENTINEL-1',
                    'productType': 'SLC'
                },
                'S1_SAR_OCN': {
                    'collection': 'SENTINEL-1',
                    'productType': 'OCN'
                },
                
                # # Sentinel-1 RTC products
                # 'S1_RTC': {
                #     'collection': 'SENTINEL-1-RTC',
                #     'productType': 'RTC'
                # },
                
                # Sentinel-2 products  
                'S2_MSI_L1C': {
                    'collection': 'SENTINEL-2',
                    'productType': 'S2MSI1C'
                },
                'S2_MSI_L2A': {
                    'collection': 'SENTINEL-2',
                    'productType': 'S2MSI2A'
                },
                
            }
        }
    }
    
    return config

In [2]:
from eodag import EODataAccessGateway
from eodag import setup_logging
setup_logging(verbose=2)

/home/martin/Documents/ONFI/Projects/Tests/eodata-gateway/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
dag = EODataAccessGateway()
dag.available_providers()

2025-06-10 17:55:40,446 eodag.config                     [INFO    ] Loading user configuration from: /home/martin/.config/eodag/eodag.yml
2025-06-10 17:55:40,464 eodag.core                       [INFO    ] aws_eos: provider needing auth for search has been pruned because no credentials could be found
2025-06-10 17:55:40,464 eodag.core                       [INFO    ] cop_ads: provider needing auth for search has been pruned because no credentials could be found
2025-06-10 17:55:40,464 eodag.core                       [INFO    ] cop_cds: provider needing auth for search has been pruned because no credentials could be found
2025-06-10 17:55:40,465 eodag.core                       [INFO    ] meteoblue: provider needing auth for search has been pruned because no credentials could be found
2025-06-10 17:55:40,465 eodag.core                       [INFO    ] hydroweb_next: provider needing auth for search has been pruned because no credentials could be found
2025-06-10 17:55:40,465 eodag.core

['peps',
 'cop_dataspace',
 'cop_marine',
 'creodias',
 'earth_search',
 'earth_search_cog',
 'earth_search_gcs',
 'eumetsat_ds',
 'geodes',
 'planetary_computer',
 'sara',
 'theia',
 'usgs_satapi_aws']

### Tests with Opensearch

Pourquoi Opensearch vs autres servivces (OData, STAC) ?
=>Beaucoup plus de catalogues disponibles !!! dont Sentinel 1 RTC
https://documentation.dataspace.copernicus.eu/APIs/OpenSearch.html


In [4]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

True

In [5]:
config = create_opensearch_provider_config()

In [6]:
dag.update_providers_config(dict_conf=config)

In [7]:
dag.set_preferred_provider("cop_dataspace")

In [8]:
product_type = 'S2_MSI_L1C'
extent = {
    'lonmin': -105.095901,
    'lonmax': -104.977112,
    'latmin': 36.500253,
    'latmax': 36.559015
}

products_before = dag.search(
    productType=product_type,
    start='2018-05-21',
    end='2018-05-25',
    geom=extent,
)
products_before

2025-06-10 17:55:43,745 eodag.core                       [INFO    ] Searching on provider cop_dataspace
2025-06-10 17:55:43,745 eodag.search.base                [INFO    ] cop_dataspace is configured with default sorting by 'startTimeFromAscendingNode' in ascending order
2025-06-10 17:55:43,746 eodag.core                       [ERROR   ] Error while searching on provider cop_dataspace (ignored):
Traceback (most recent call last):
  File "/home/martin/Documents/ONFI/Projects/Tests/eodata-gateway/.venv/lib/python3.12/site-packages/eodag/api/core.py", line 1856, in _do_search
    res, nb_res = search_plugin.query(prep, **kwargs)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/martin/Documents/ONFI/Projects/Tests/eodata-gateway/.venv/lib/python3.12/site-packages/eodag/plugins/search/qssearch.py", line 778, in query
    qp, qs = self.build_query_string(product_type, keywords)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/martin/Documents/ONFI

{'union': ' or ', 'wrapper': '{}', 'operations': {'and': ["Collection/Name eq '{collection}'", "OData.CSC.Intersects(area=geography'{geometry#to_ewkt}')", "Attributes/OData.CSC.StringAttribute/any(att:att/Name eq 'productType' and att/OData.CSC.StringAttribute/Value eq '{productType}')", "Attributes/OData.CSC.StringAttribute/any(att:att/Name eq 'platformShortName' and att/OData.CSC.StringAttribute/Value eq '{platform}')", "Attributes/OData.CSC.StringAttribute/any(att:att/Name eq 'platformSerialIdentifier' and att/OData.CSC.StringAttribute/Value eq '{platformSerialIdentifier}')", "Attributes/OData.CSC.IntegerAttribute/any(att:att/Name eq 'spatialResolution' and att/OData.CSC.StringAttribute/Value eq '{resolution}')", "Attributes/OData.CSC.StringAttribute/any(att:att/Name eq 'authority' and att/OData.CSC.StringAttribute/Value eq '{organisationName}')", "Attributes/OData.CSC.IntegerAttribute/any(att:att/Name eq 'orbitNumber' and att/OData.CSC.IntegerAttribute/Value eq {orbitNumber})", "At

2025-06-10 17:55:44,361 eodag.core                       [WARNING ] No result could be obtained from provider peps, we will try to get the data from another provider
2025-06-10 17:55:44,362 eodag.core                       [INFO    ] Searching on provider creodias
2025-06-10 17:55:44,363 eodag.search.base                [INFO    ] creodias is configured with default sorting by 'startTimeFromAscendingNode' in ascending order
2025-06-10 17:55:44,368 eodag.search.qssearch            [INFO    ] Sending search request: https://datahub.creodias.eu/odata/v1/Products?$filter=Collection/Name%20eq%20%27SENTINEL-2%27%20and%20OData.CSC.Intersects%28area=geography%27SRID=4326%3BPOLYGON%20%28%28-105.0959%2036.5003%2C%20-105.0959%2036.5590%2C%20-104.9771%2036.5590%2C%20-104.9771%2036.5003%2C%20-105.0959%2036.5003%29%29%27%29%20and%20Attributes/OData.CSC.StringAttribute/any%28att:att/Name%20eq%20%27productType%27%20and%20att/OData.CSC.StringAttribute/Value%20eq%20%27S2MSI1C%27%29%20and%20ContentDate/S

{'union': ' or ', 'wrapper': '{}', 'operations': {'and': ["Collection/Name eq '{collection}'", "OData.CSC.Intersects(area=geography'{geometry#to_ewkt}')", "Attributes/OData.CSC.StringAttribute/any(att:att/Name eq 'productType' and att/OData.CSC.StringAttribute/Value eq '{productType}')", "Attributes/OData.CSC.StringAttribute/any(att:att/Name eq 'platformShortName' and att/OData.CSC.StringAttribute/Value eq '{platform}')", "Attributes/OData.CSC.StringAttribute/any(att:att/Name eq 'platformSerialIdentifier' and att/OData.CSC.StringAttribute/Value eq '{platformSerialIdentifier}')", "Attributes/OData.CSC.IntegerAttribute/any(att:att/Name eq 'spatialResolution' and att/OData.CSC.StringAttribute/Value eq '{resolution}')", "Attributes/OData.CSC.StringAttribute/any(att:att/Name eq 'authority' and att/OData.CSC.StringAttribute/Value eq '{organisationName}')", "Attributes/OData.CSC.IntegerAttribute/any(att:att/Name eq 'orbitNumber' and att/OData.CSC.StringAttribute/Value eq '{orbitNumber}')", "A

SearchResult([EOProduct(id=S2A_MSIL1C_20180524T174051_N0500_R098_T13SDA_20230721T061025, provider=creodias),
              EOProduct(id=S2A_MSIL1C_20180524T174051_N0500_R098_T13SEA_20230721T061025, provider=creodias)])

In [ ]:
# Save configuration to YAML file
import yaml

with open('eodag_opensearch_config.yml', 'w') as f:
    yaml.dump(config, f, default_flow_style=False, indent=2)

print("Configuration saved to eodag_opensearch_config.yml")
print(f"Total products configured: {len(config['cop_dataspace']['products'])}")